<a href="https://colab.research.google.com/github/kulyulinanina/-3-/blob/main/HW_4_Nina_Kulyulina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install -q -U kaggle_environments

In [131]:
import random
from kaggle_environments import make, evaluate
import pandas as pd

In [ ]:
# Агент 1: Всегда выбирает "камень" (0)
def rock_agent(observation, configuration):
    return 0

In [ ]:
# Агент 2: Всегда выбирает "ножницы" (2)
def scissors_agent(observation, configuration):
    return 2

In [ ]:
# Агент 3: Всегда выбирает "бумагу" (1)
def paper_agent(observation, configuration):
    return 1

In [ ]:
# Агент 4: Выбирает случайное действие
def random_agent(observation, configuration):
    return random.choice([0, 1, 2])

In [ ]:
# Агент 5: Копирует действие противника
def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.choice([0, 1, 2])

In [ ]:
# Агент 6: Повторяет свои предыдущие действия
def repeat_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastAction
    else:
        return random.choice([0, 1, 2])

In [106]:
# Агент 7: Пытается побить последний ход противника
def beat_last_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % 3
    else:
        return random.choice([0, 1, 2])

In [94]:
# Агент 8: Использует статистику прошлых ходов противника
def stats_agent(observation, configuration):
    if observation.step > 0:
        print(observation.lastOpponentAction)
        counts = [0, 0, 0]
        for action in observation.lastOpponentActions:
            counts[action] += 1
        return (counts.index(max(counts)) + 1) % 3
    else:
        return random.choice([0, 1, 2])

In [ ]:
# Агент 9: Меняет действия циклически
def cycle_agent(observation, configuration):
    return observation.step % 3

In [ ]:
# Агент 10: Использует случайность с повышенной вероятностью камня
def biased_random_agent_0(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.6, 0.2, 0.2])[0]

In [ ]:
# Агент 11: Использует случайность с повышенной вероятностью ножниц
def biased_random_agent_2(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.2, 0.2, 0.6])[0]

In [ ]:
# Агент 12: Использует случайность с повышенной вероятностью бумаги
def biased_random_agent_1(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.2, 0.6, 0.2])[0]

In [ ]:
# Агент 13: Играет наоборот против оппонента
def reverse_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % 3
    else:
        return random.choice([0, 1, 2])

In [ ]:
# Агент 14: Использует стратегию "предсказание на основе распределения вероятностей"
def probability_predictor_agent(observation, configuration):
    if observation.step > 0:
        # Считаем частоту ходов противника
        counts = [0, 0, 0]
        for action in observation.lastOpponentActions:
            counts[action] += 1
        total = sum(counts)
        probabilities = [count / total for count in counts]

        # Выбираем действие с наибольшей вероятностью выигрыша
        predicted_action = probabilities.index(max(probabilities))
        return (predicted_action + 1) % 3
    else:
        return random.choice([0, 1, 2])

In [ ]:
# Создаем окружающую среду
env = make("rps", configuration={"episodeSteps": 100})

In [ ]:
# Список агентов
agents = [rock_agent, scissors_agent, paper_agent, random_agent, copy_opponent_agent,
          repeat_agent, beat_last_agent, stats_agent, cycle_agent,
          biased_random_agent_0, biased_random_agent_2, biased_random_agent_1,
          reverse_agent, probability_predictor_agent]

In [124]:
# Турнир между агентами
results = {}
for i, agent_1 in enumerate(agents):
    for j, agent_2 in enumerate(agents):
        if i < j:
            outcomes = evaluate("rps", [agent_1, agent_2], configuration={"episodeSteps": 100, "tieRewardThreshold": 1})
            results[f"Agent {i + 1} vs Agent {j + 1}"] = sum([o[0] for o in outcomes if o is not None and o[0] is not None])

In [133]:
tournament_table = pd.DataFrame(index=[f"Agent {i+1}" for i in range(len(agents))],
                                columns=[f"Agent {i+1}" for i in range(len(agents))])

for match, score in results.items():
    agent_1_num, agent_2_num = [int(s.split(" ")[1]) for s in match.split(" vs ")]
    tournament_table.loc[f"Agent {agent_1_num}", f"Agent {agent_2_num}"] = score
    tournament_table.loc[f"Agent {agent_2_num}", f"Agent {agent_1_num}"] = -score
tournament_table = tournament_table.fillna('')

**Инструкция по чтению датафрейма:**


- Значение в ячейке показывает суммарный результат агента в строке в играх против агента в столбце.
- Положительное значение означает, что агент в строке выиграл больше игр, чем проиграл.
- Отрицательное значение означает, что агент в строке проиграл больше игр, чем выиграл.
- Ноль означает, что агенты сыграли вничью.

In [136]:
tournament_table

,Agent 1,Agent 2,Agent 3,Agent 4,Agent 5,Agent 6,Agent 7,Agent 8,Agent 9,Agent 10,Agent 11,Agent 12,Agent 13
Agent 1,,99.0,-99.0,6.0,1.0,-98.0,1,0,-4.0,40.0,-45.0,99.0,1
Agent 2,-99.0,,99.0,-4.0,-1.0,-97.0,1,0,-45.0,0,43.0,99.0,1
Agent 3,99.0,-99.0,,13.0,0,-99.0,1,0,44.0,-48.0,-6.0,97.0,1
Agent 4,-6.0,4.0,-13.0,,-17.0,-5.0,1,-2.0,-17.0,9.0,3.0,7.0,1
Agent 5,-1.0,1.0,0,17.0,,-49.0,1,-98.0,3.0,6.0,-4.0,50.0,1
Agent 6,98.0,97.0,99.0,5.0,49.0,,1,0,18.0,23.0,26.0,-50.0,1
Agent 7,-1,-1,-1,-1,-1,-1,,0,0,0,0,0,0
Agent 8,0,0,0,2.0,98.0,0,0,,1.0,1.0,0,-97.0,1
Agent 9,4.0,45.0,-44.0,17.0,-3.0,-18.0,0,-1.0,,10.0,-2.0,16.0,1
Agent 10,-40.0,0,48.0,-9.0,-6.0,-23.0,0,-1.0,-10.0,,32.0,30.0,1
